In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from scipy import io
import gc
import pickle
from tqdm import tqdm
from concurrent import futures
import time
import warnings
from scipy import io
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

def train_test_split(matrix, ratio=0.2):
    shape_row, shape_col = matrix.shape
    row = []
    col = []
    data = []
    for i in tqdm(range(shape_row)):
        nonzero = matrix[i].nonzero()[1]
        sample_len = int(len(nonzero) * ratio)
        sample = np.random.choice(nonzero, sample_len, replace=False)
        row.extend([i] * sample_len)
        col.extend(sample)
        data.extend([1] * sample_len)
    test_matrix = sparse.csr_matrix((data, (row, col)), shape=(shape_row, shape_col))
    train_matrix = matrix - test_matrix
    return test_matrix, train_matrix

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

/home/liuy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
douban_mat = io.loadmat('douban.mat')
matrix_UM = douban_mat['relation'][0, 0]
matrix_UU = douban_mat['relation'][0, 1]
matrix_UG = douban_mat['relation'][0, 2]
matrix_UL = douban_mat['relation'][0, 3]
matrix_MD = douban_mat['relation'][0, 4]
matrix_MA = douban_mat['relation'][0, 5]
matrix_MT = douban_mat['relation'][0, 6]

test = load_obj('douban/20/4/test.mtx')
train = load_obj('douban/20/4/train.mtx')
matrix_UGUM = matrix_UG * matrix_UG.T * train
matrix_UMUM = train * train.T * train
matrix_UMDMUM = train * matrix_MD * matrix_MD.T * train.T * train
matrix_UMAMUM = train * matrix_MA * matrix_MA.T * train.T * train
matrix_UMTMUM = train * matrix_MT * matrix_MT.T * train.T * train

user_shape, item_shape = train.shape
train_row, train_col = train.nonzero()
test_row, test_col = test.nonzero()

train_rates = np.array(matrix_UM[train_row, train_col])[0]
test_rates = np.array(matrix_UM[test_row, test_col])[0]

train_size = len(train_row)
test_size = len(test_row)

batchs = np.array(range(train_size))
np.random.seed(2017)
np.random.shuffle(batchs)

batch_size = 10000
begin = 0
end = batch_size

matrix_UGUM = matrix_UGUM.todense()
matrix_UMUM = matrix_UMUM.todense()
matrix_UMDMUM = matrix_UMDMUM.todense()
matrix_UMAMUM = matrix_UMAMUM.todense()
matrix_UMTMUM = matrix_UMTMUM.todense()

tf_UGUM = tf.placeholder(tf.float64, [user_shape, item_shape])
tf_UMUM = tf.placeholder(tf.float64, [user_shape, item_shape])
tf_UMDMUM = tf.placeholder(tf.float64, [user_shape, item_shape])
tf_UMAMUM = tf.placeholder(tf.float64, [user_shape, item_shape])
tf_UMTMUM = tf.placeholder(tf.float64, [user_shape, item_shape])

tensor_train_rows = tf.placeholder(tf.int32, [None])
tensor_train_cols = tf.placeholder(tf.int32, [None])
truth_index = tf.placeholder(tf.int32, [None])
test_truth_index = tf.placeholder(tf.int32, [None])

train_true_rates = tf.expand_dims(tf.constant(train_rates), 1)
test_true_rates = tf.expand_dims(tf.constant(test_rates), 1)

user_input_UMUM = tf.nn.embedding_lookup(tf_UMUM, tensor_train_rows)
item_input_UMUM = tf.nn.embedding_lookup(tf.transpose(tf_UMUM), tensor_train_cols)

user_input_UGUM = tf.nn.embedding_lookup(tf_UGUM, tensor_train_rows)
item_input_UGUM = tf.nn.embedding_lookup(tf.transpose(tf_UGUM), tensor_train_cols)

user_input_UMDMUM = tf.nn.embedding_lookup(tf_UMDMUM, tensor_train_rows)
item_input_UMDMUM = tf.nn.embedding_lookup(tf.transpose(tf_UMDMUM), tensor_train_cols)

user_input_UMAMUM = tf.nn.embedding_lookup(tf_UMAMUM, tensor_train_rows)
item_input_UMAMUM = tf.nn.embedding_lookup(tf.transpose(tf_UMAMUM), tensor_train_cols)

user_input_UMTMUM = tf.nn.embedding_lookup(tf_UMTMUM, tensor_train_rows)
item_input_UMTMUM = tf.nn.embedding_lookup(tf.transpose(tf_UMTMUM), tensor_train_cols)

n_hidden_1 = 128
n_embedding = 64


user_embedding_UMUM = tf.layers.dense(tf.layers.dense(user_input_UMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)
user_embedding_UGUM = tf.layers.dense(tf.layers.dense(user_input_UGUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)             
user_embedding_UMDMUM = tf.layers.dense(tf.layers.dense(user_input_UMDMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)                             
user_embedding_UMAMUM = tf.layers.dense(tf.layers.dense(user_input_UMAMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)                          
user_embedding_UMTMUM = tf.layers.dense(tf.layers.dense(user_input_UMTMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)

item_embedding_UMUM = tf.layers.dense(tf.layers.dense(item_input_UMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)                                     
item_embedding_UGUM = tf.layers.dense(tf.layers.dense(item_input_UGUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)                                   
item_embedding_UMDMUM = tf.layers.dense(tf.layers.dense(item_input_UMDMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)
item_embedding_UMAMUM = tf.layers.dense(tf.layers.dense(item_input_UMAMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)                                      
item_embedding_UMTMUM = tf.layers.dense(tf.layers.dense(item_input_UMTMUM, units=n_hidden_1, activation=tf.nn.tanh), units=n_embedding, activation=tf.nn.tanh)

W_u_init = tf.placeholder(tf.float64, [user_shape, n_embedding])
W_i_init = tf.placeholder(tf.float64, [item_shape, n_embedding])

W_u = tf.Variable(W_u_init)
W_i = tf.Variable(W_i_init)

W_u_train = tf.nn.embedding_lookup(W_u, tensor_train_rows)
W_i_train = tf.nn.embedding_lookup(W_i, tensor_train_cols)

user_input = tf.stack((user_embedding_UGUM, user_embedding_UMUM, user_embedding_UMDMUM, user_embedding_UMAMUM, user_embedding_UMTMUM))
item_input = tf.stack((item_embedding_UGUM, item_embedding_UMUM, item_embedding_UMDMUM, item_embedding_UMAMUM, item_embedding_UMTMUM))

user_val = tf.transpose(tf.nn.softmax(tf.transpose(tf.reduce_sum(tf.multiply(user_input, tf.expand_dims(W_u_train, 0)), 2))))
item_val = tf.transpose(tf.nn.softmax(tf.transpose(tf.reduce_sum(tf.multiply(item_input, tf.expand_dims(W_i_train, 0)), 2))))

user_feature = tf.reduce_sum(tf.multiply(tf.expand_dims(user_val, 2), user_input), 0)
item_feature = tf.reduce_sum(tf.multiply(tf.expand_dims(item_val, 2), item_input), 0)

FM_input = tf.stack((user_feature, item_feature))
W_ui = tf.Variable(np.zeros([1, n_embedding]))
input_val = tf.transpose(tf.nn.softmax(tf.transpose(tf.reduce_sum(tf.multiply(FM_input, tf.expand_dims(W_ui, 0)), 2))))
user_items = tf.reduce_sum(tf.multiply(tf.expand_dims(input_val, 2), FM_input), 0)

# D = n_embedding * 2
D = n_embedding
F = 30

Bias_fm = tf.Variable(np.zeros([1]))
# W_fm = tf.Variable(np.zeros([D, 1]))
# V = tf.Variable(np.zeros([D, F]))
W_fm = tf.Variable(tf.random_normal([D, 1], stddev=0.001, dtype=tf.float64, seed=2018))
V = tf.Variable(tf.random_normal([D, F], stddev=0.001, dtype=tf.float64, seed=2018))

reg = tf.nn.l2_loss(Bias_fm) + tf.nn.l2_loss(W_fm) + tf.nn.l2_loss(V)

FM_res = tf.matmul(user_items, W_fm) + 0.5 * tf.expand_dims(tf.reduce_sum(tf.matmul(user_items, V), 1), 1) \
            - 0.5 * tf.expand_dims(tf.reduce_sum(tf.matmul(tf.square(user_items), tf.square(V)), 1), 1) + tf.expand_dims(Bias_fm, 1)
ground_truth = tf.nn.embedding_lookup(train_true_rates, truth_index)
loss = tf.reduce_sum(tf.square(FM_res - ground_truth), 0) 
gds = tf.train.AdamOptimizer(0.001).minimize(loss)
t_labels = tf.nn.embedding_lookup(test_true_rates, test_truth_index)
rmse = tf.reduce_sum(tf.square(FM_res - t_labels), 0)
mae = tf.reduce_sum(tf.abs(FM_res - t_labels), 0)

u_init = np.zeros([user_shape, n_embedding]).astype(np.float64)
i_init = np.zeros([item_shape, n_embedding]).astype(np.float64)
sess = tf.Session()
sess.run(tf.global_variables_initializer(), {W_u_init: u_init, W_i_init: i_init})

for _ in tqdm(range(410)):
    if _ % 100 == 0 and _ >= 100:
        df = pd.DataFrame(sess.run(W_u))
        print (df.sum().sum())
        err = 0
        merr = 0
        test_len = len(test_row)
        split = []
        test_round = 30
        starttime = datetime.datetime.now()
        for i in range(test_round + 1):
            split.append(int(test_len * i / test_round))
        for i in range(test_round):
            test_sample_rows = np.array(test_row[split[i]:split[i + 1]]).astype(np.int)
            test_sample_cols = np.array(test_col[split[i]:split[i + 1]]).astype(np.int)
            test_sample = np.array(range(split[i], split[i + 1])).astype(np.int)
            targs = {
                tf_UMDMUM: matrix_UMDMUM,
                tf_UMAMUM: matrix_UMAMUM,
                tf_UMTMUM: matrix_UMTMUM,
                tf_UGUM: matrix_UGUM,
                tf_UMUM: matrix_UMUM,
                tensor_train_rows: test_sample_rows, 
                tensor_train_cols: test_sample_cols, 
                test_truth_index: test_sample
            }
            err += sess.run(rmse, targs)
            merr += sess.run(mae, targs)
        endtime = datetime.datetime.now()
        print ('time: ', (endtime - starttime).seconds)
        print ('rmse: ', np.sqrt(err / test_size))
        print ('mae: ', merr / test_size)
    sample_index = batchs[begin:end]
    sample_rows = np.array(train_row[sample_index]).astype(np.int)
    sample_cols = np.array(train_col[sample_index]).astype(np.int)
    if end + batch_size > train_size:
        begin = 0
        end = batch_size
    else:
        begin += batch_size
        end += batch_size
    args = {
        tf_UMDMUM: matrix_UMDMUM,
        tf_UMAMUM: matrix_UMAMUM,
        tf_UMTMUM: matrix_UMTMUM,
        tf_UGUM: matrix_UGUM,
        tf_UMUM: matrix_UMUM,
        tensor_train_rows: sample_rows,
        tensor_train_cols: sample_cols,
        truth_index: sample_index
    }
    l = sess.run(gds, args)
sess.close()
tf.reset_default_graph()

 24%|██▍       | 100/410 [20:29<1:03:30, 12.29s/it]

-364.3842147941423
time:  911
rmse:  [0.73650969]
mae:  [0.57710008]


 49%|████▉     | 200/410 [56:38<59:28, 16.99s/it]  

-487.40030336283667
time:  899
rmse:  [0.73349738]
mae:  [0.57545361]


 73%|███████▎  | 300/410 [1:32:11<33:48, 18.44s/it]  

-473.1459140560952
time:  888
rmse:  [0.73166543]
mae:  [0.57461425]


 98%|█████████▊| 400/410 [2:07:46<03:11, 19.17s/it]

-390.26265632095703
time:  918
rmse:  [0.72825548]
mae:  [0.56844718]


100%|██████████| 410/410 [2:25:10<00:00, 21.25s/it]
